<a href="https://colab.research.google.com/github/rdiazrincon/SLEAP_Data/blob/master/analysis/prediction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning for behavioral prediction of differences in dominant vs subordinate mice during reward competition 

### (Optional) Installing required packages

**Note:** If you are runnning this Notebook from Colab most packages will be already installed. But please do so if you are running locally.

In [1]:
# !pip install h5py
# !pip install matplotlib
# !pip install seaborn
# !pip install pandas
# !pip install sklearn
# !pip install numpy
# !pip install scipy
# !pip install tables

### Importing packages

In [2]:
import h5py
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from scipy.interpolate import interp1d

### Getting the data from Github 

**Note:** Run this if you are using Colab. It will save the labeled data in your current directory.

In [3]:
# !wget -O prediction_analysis.h5 https://github.com/rdiazrincon/SLEAP_Data/blob/master/analysis/data/prediction_analysis.h5

### Setting up the paths for labeled data and reward competition info

In [4]:
path = "data/prediction_analysis.h5"
intervals = "data/WL_info_by_timestamp.csv"

## Exploring the data

In [5]:
# Opening the H5 file
with h5py.File(path, "r") as f:
    dataset_keys = list(f.keys())
    tracks = f["tracks"][:].T
    bodyparts = [n.decode() for n in f["node_names"][:]]

print(f"Filename:{path}")
print(f"Array keys{dataset_keys}:")
print(f"Behavioral data shape:{tracks.shape}")
print(f"Body parts{bodyparts}")

OSError: Unable to open file (file signature not found)

## Understanding the data 

We only work with the tracks array. It contains information regarding Number of Frames, Number of Bodyparts, Number of Coordinates and Number of instances.


In [ ]:
# Here's what each dimension of the matrix's shape means
n_frames, n_bodyparts, n_coordinates, n_instances = tracks.shape

print(f"Number of Frames: {n_frames}")
print(f"Number of Bodyparts: {n_bodyparts}")
print(f"Number of Coordinates: {n_coordinates}")
print(f"NUmber of Instances: {n_instances}")

### Exploring (and making sense out of) the data

- The tracks array contains all of the information we need. 
- The first dimension of the array will provide us with information regarding the frames labeled in the video. 
- The second dimension relates to each body part (6 in our case). Note that, in the example below, we get data from the forehead by accesing position 5 in the array. 
- The third dimension relates to the X and Y coordinates and the fourth dimension to the number if animals/mice/instances


#### First frame 

In [ ]:
print(tracks[0])

#### Forehead coordinates for both mouse in Frame 1

**Note:** You can find the forehead data in position 5 of the tracks array. In general, the "bodyparts" array contains information regarding the position of bodyparts.
 

In [ ]:
print(tracks[0][5])

#### Forehead X-coordinate for both mouse in Frame 1

In [ ]:
print(tracks[0][5][0])

#### Forehead Y-coordinate for mouse 2 in Frame 1.

In [ ]:
print(tracks[0][5][0][1]) 

## Now let's mess around with the data ...

### Fill-in missing values

We won't drop the NaN values but will fill them with Non Zero values so the analysis is easier to make

In [ ]:
def fill_missing(Y, kind="linear"):
    """Fills missing values independently along each dimension after the first."""

    # Store initial shape.
    initial_shape = Y.shape

    # Flatten after first dim.
    Y = Y.reshape((initial_shape[0], -1))

    # Interpolate along each slice.
    for i in range(Y.shape[-1]):
        y = Y[:, i]

        # Build interpolant.
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)

        # Fill missing
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        
        # Fill leading or trailing NaNs with the nearest non-NaN values
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])

        # Save slice
        Y[:, i] = y

    # Restore to initial shape.
    Y = Y.reshape(initial_shape)

    return Y

After filling out the NaN values with None Zero values, the array maintains the same shape

In [ ]:
tracks = fill_missing(tracks)
print(tracks.shape)

### Reading and handling information related to Reward Competition

The information in the Frames column relates to the frames that make up a Trial. We will be reading a file to get this information


In [ ]:
df_trials = pd.read_csv(intervals)['Frame']
df_trials

### Converting the trials DataFrame to an array

Using the dash to separate each trial in an array

In [ ]:
trials_array_separated = [trial.split("-") for trial in df_trials] # Returns a list of lists
print(trials_array_separated)

Looping through the nested array, converting each (str) element to int and appending the elements in a new array 


In [ ]:
trials_array = [int(j) for i in range(len(trials_array_separated)) for j in trials_array_separated[i]]
print(trials_array)

# Leaving this here for future reference
# for i in range(len(trials_array_separated)):
#     for j in trials_array_separated[i]:
#         trials_array.append(eval(j))
        
# trials_array

## Getting information on mice coordinates based in trials 

For each body part, we loop through the set of frames that comprise a single trial. We repeat for all the trials in the competition (20).
Each video has 30 frames per second(FPS) and each trial lasts 10 second.

In [ ]:
LEFT_EAR_INDEX = 0
RIGHT_EAR_INDEX = 1
NOSE_INDEX = 2
TAIL_BASE_INDEX = 3
THORAX_INDEX = 4
FOREHEAD_INDEX = 5

TRIAL_LENGTH_SECONDS = 10
N_TRIALS = 20
FPS = 30

time_window_seconds = 5

frames_per_trial = FPS * TRIAL_LENGTH_SECONDS
time_window_frames = FPS * time_window_seconds

left_ear_loc_aux = []
right_ear_loc_aux = []
nose_loc_aux = []
tail_base_loc_aux = []
thorax_loc_aux = []
forehead_loc_aux = []

Using the onset information to target a different set of frames. Now we are looking at the frames that occur 30 seconds prior to the tone onset and 30 seconds before the tone onset. That is, we will be looking at time_window_frames frames before and after

In [ ]:
it = iter(trials_array)
trials_array_onset = []
for i, j in zip(it, it):
    trials_array_onset.append(i - time_window_frames)
    trials_array_onset.append(j)

In [ ]:
print(trials_array_onset)

In [ ]:
print(trials_array)     

Using the trials_array to loop through the intervals (first_frame_in_trial:last_frame_in_trial) and fill the vector with the coordinates corresponding for both mice and their body parts  


In [ ]:
iterator = iter(trials_array_onset)
for first_frame_in_trial, last_frame_in_trial in zip(iterator, iterator):
    # print(len(tracks[first_frame_in_trial:last_frame_in_trial, RIGHT_EAR_INDEX, :, :]))
    left_ear_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, LEFT_EAR_INDEX, :, :])
    right_ear_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, RIGHT_EAR_INDEX, :, :])
    nose_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, NOSE_INDEX, :, :])
    tail_base_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, TAIL_BASE_INDEX, :, :])
    thorax_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, THORAX_INDEX, :, :])
    forehead_loc_aux.append(tracks[first_frame_in_trial:last_frame_in_trial, FOREHEAD_INDEX, :, :])

frames_in_trial = last_frame_in_trial - first_frame_in_trial

In [ ]:
array_shape = np.asarray(left_ear_loc_aux).shape
print(f"Old shape: {array_shape}")
left_ear_loc = np.asarray(left_ear_loc_aux).reshape(N_TRIALS * frames_in_trial , n_coordinates, n_instances)
print(f"New shape: {left_ear_loc.shape}")
right_ear_loc = np.asarray(right_ear_loc_aux).reshape(N_TRIALS * frames_in_trial, n_coordinates, n_instances)
nose_loc = np.asarray(nose_loc_aux).reshape(N_TRIALS * frames_in_trial, n_coordinates, n_instances)
tail_base_loc = np.asarray(tail_base_loc_aux).reshape(N_TRIALS * frames_in_trial, n_coordinates, n_instances)
thorax_loc = np.asarray(thorax_loc_aux).reshape(N_TRIALS * frames_in_trial, n_coordinates, n_instances)
forehead_loc = np.asarray(forehead_loc_aux).reshape(N_TRIALS * frames_in_trial, n_coordinates, n_instances)

### Converting our data into a Tabular format

In [ ]:
# There is definitely an easier way of doing this but I don't have the time now

bp = ['left_ear', 'right_ear', 'thorax', 'tail', 'nose']
coord = ['x', 'y']
for i in range(1):
    for j in range(1):
        df_bla = {
            'm{j}_{bp[{0}]}_{i}': left_ear_loc[:, i, j]
        }

df = pd.DataFrame({
    'm1_left_ear_x': left_ear_loc[:, 0, 0], 
    'm1_left_ear_y': left_ear_loc[:, 1, 0], 
    "m2_left_ear_x": left_ear_loc[:, 0, 1],
    "m2_left_ear_y": left_ear_loc[:, 1, 1],

    'm1_right_ear_x': right_ear_loc[:, 0, 0], 
    'm1_right_ear_y': right_ear_loc[:, 1, 0], 
    "m2_right_ear_x": right_ear_loc[:, 0, 1],
    "m2_right_ear_y": right_ear_loc[:, 1, 1],
    
    'm1_nose_x': nose_loc[:, 0, 0], 
    'm1_nose_y': nose_loc[:, 1, 0], 
    "m2_nose_x": nose_loc[:, 0, 1],
    "m2_nose_y": nose_loc[:, 1, 1],

    'm1_tail_base_x': tail_base_loc[:, 0, 0], 
    'm1_tail_base_y': tail_base_loc[:, 1, 0], 
    "m2_tail_base_x": tail_base_loc[:, 0, 1],
    "m2_tail_base_y": tail_base_loc[:, 1, 1],

    'm1_thorax_x': thorax_loc[:, 0, 0], 
    'm1_thorax_y': thorax_loc[:, 1, 0], 
    "m2_thorax_x": thorax_loc[:, 0, 1],
    "m2_thorax_y": thorax_loc[:, 1, 1],

    'm1_forehead_x': forehead_loc[:, 0, 0], 
    'm1_forehead_y': forehead_loc[:, 1, 0], 
    "m2_forehead_x": forehead_loc[:, 0, 1],
    "m2_forehead_y": forehead_loc[:, 1, 1],
    })
df

### Further exploration of the data

In [ ]:
df.describe()

In [ ]:
df.sample()

In [ ]:
df.value_counts()

In [ ]:
df2 = pd.read_csv(intervals)['Winner (1:M_1,0:M_2,2:Tie)']
df2

In [ ]:
features_array = df2.to_numpy().repeat(frames_in_trial)
print(features_array.shape)

### Running Random Forest for prediction

In [ ]:
X = df
Y = features_array

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

In [ ]:
model = RandomForestClassifier(n_estimators = 10, random_state = 30)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
df_see_frames = X_test.index
df_see_frames

In [ ]:
prediction_test = model.predict(X_test)
prediction_test

In [ ]:
prediction_probability = model.predict_proba(X_test)
prediction_probability

In [ ]:
print (f"Accuracy: {metrics.accuracy_score(y_test, prediction_test):.2f}")

In [ ]:
print(model.score(X_test, y_test))

In [ ]:
X_test

In [ ]:
confusion_matrix_table = pd.DataFrame({"Frame": df_see_frames, "Predicted Class Label": np.array(prediction_test), "Actual Class Label": np.array(y_test),})
confusion_matrix_table = confusion_matrix_table.sort_values(by='Frame', ascending=True)
confusion_matrix_table

In [ ]:
feature_list = list(X.columns)
feature_list

In [ ]:
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
feature_imp

## Plotting the Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, prediction_test)
sns.set(rc={'figure.figsize':(10,10)}, font_scale=1.5)
# plot = sns.heatmap(cm, annot=True, square=True, linewidths=.5, cmap="YlOrBr", fmt='d')
plot = sns.heatmap(cm, annot=True, square=True, linewidths=.5, cmap="YlGnBu", fmt='d')
plot.set(xlabel='Predicted Frames', ylabel='True Frames')

In [ ]:
sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

In [ ]:
# Figure 1
plt.figure()

plt.plot(nose_loc[:,0,0], 'r',label='Mouse_1')
plt.plot(nose_loc[:,0,1], 'b',label='Mouse_2')
# plt.plot(-1*nose_loc[:,1,0], 'r')
# plt.plot(-1*nose_loc[:,1,1], 'b')

plt.legend(loc="center right")

plt.title('Nose location')

# Figure 2
plt.figure(figsize=(15,15))

plt.plot(nose_loc[:,0,0],nose_loc[:,1,0], 'r',label='Mouse_1')
plt.plot(nose_loc[:,0,1],nose_loc[:,1,1], 'b',label='Mouse_2')
plt.legend()

plt.xlim(0,1024)
plt.xticks([])

plt.ylim(0,1024)
plt.yticks([])
plt.title('Nose tracks across frames')

## More advanced visualizations (In Progress)

For some additional analysis, we'll first smooth and differentiate the data with a Savitzky-Golay filter to extract velocities of each joint.

In [ ]:
from scipy.signal import savgol_filter

def smooth_diff(node_loc, win=25, poly=3):
    """
    node_loc is a [frames, 2] array
    
    win defines the window to smooth over
    
    poly defines the order of the polynomial
    to fit with
    
    """
    node_loc_vel = np.zeros_like(node_loc)
    
    for c in range(node_loc.shape[-1]):
        node_loc_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv=1)
    
    node_vel = np.linalg.norm(node_loc_vel,axis=1)

    return node_vel

There are two mice. Let's get results for each separately.

In [ ]:
thx_vel_mouse1 = smooth_diff(nose_loc[:, :, 0].astype('float64'))
thx_vel_mouse2 = smooth_diff(nose_loc[:, :, 1].astype('float64'))

### Visualizing thorax x-y dynamics and velocity for Mouse_1

In [ ]:
fig = plt.figure(figsize=(15,7))
ax1 = fig.add_subplot(211)
ax1.plot(thorax_loc[:, 0, 0], 'k', label='x')
ax1.plot(-1*thorax_loc[:, 1, 0], 'k', label='y')
ax1.legend()
ax1.set_xticks([])
ax1.set_title('Thorax')

ax2 = fig.add_subplot(212, sharex=ax1)
# Using the Transpose of the Matrix
ax2.imshow(thx_vel_mouse1[:,np.newaxis].T, aspect='auto', vmin=0, vmax=10)
ax2.set_yticks([])
ax2.set_title('Velocity')

### Visualize thorax colored by magnitude of Mouse speed

In [ ]:
fig = plt.figure(figsize=(15,6))
ax1 = fig.add_subplot(121)
ax1.plot(thorax_loc[:, 0, 0], thorax_loc[:, 1, 0], 'k')
ax1.set_xlim(0,1024)
ax1.set_xticks([])
ax1.set_ylim(0,1024)
ax1.set_yticks([])
ax1.set_title('Thorax tracks')

kp = thx_vel_mouse1 
vmin = 0
vmax = 10

ax2 = fig.add_subplot(122)
ax2.scatter(thorax_loc[:,0,0], thorax_loc[:,1,0], c=kp, s=4, vmin=vmin, vmax=vmax)
ax2.set_xlim(0,1024)
ax2.set_xticks([])
ax2.set_ylim(0,1024)
ax2.set_yticks([])
ax2.set_title('Thorax tracks colored by magnitude of Mouse speed')

### Find covariance in thorax velocities between Mouse_1 and Mouse_2

In [ ]:
def corr_roll(datax, datay, win):
    """
    datax, datay are the two timeseries to find correlations between
    
    win sets the number of frames over which the covariance is computed
    
    """
    
    s1 = pd.Series(datax)
    s2 = pd.Series(datay)
    
    return np.array(s2.rolling(win).corr(s1))

In [ ]:
win = 50

cov_vel = corr_roll(thx_vel_mouse1, thx_vel_mouse2,win)

fig, ax = plt.subplots(2, 1, sharex=True, figsize=(15,6))
ax[0].plot(thx_vel_mouse1, 'y', label='Mouse_1')
ax[0].plot(thx_vel_mouse2, 'g', label='Mouse_2')
ax[0].legend()
ax[0].set_title('Forward Velocity')

ax[1].plot(cov_vel, 'c', markersize=1)
ax[1].set_ylim(-1.2, 1.2)
ax[1].set_title('Covariance')

fig.tight_layout()

### Todo:
- Use frame rate (30) to correlate have an intuition of which set of frames correspond to which minute/second in the video

## Clustering

For an example of clustering the data, we'll

1. extract joint velocities for each joint,
2. run k-means on the velocities from each frame.


In [ ]:
def instance_node_velocities(instance_idx):
    mouse_node_tracks = tracks[:, :, :, instance_idx]
    mouse_node_velocities = np.zeros((n_frames, n_bodyparts))

    for n in range(0, n_bodyparts):
        mouse_node_velocities[:, n] = smooth_diff(mouse_node_tracks[:, n, :])
    
    return mouse_node_velocities

In [ ]:
def plot_instance_node_velocities(instance_idx, node_velocities):
    plt.figure(figsize=(20,8))
    plt.imshow(node_velocities.T, aspect='auto', vmin=0, vmax=20, interpolation="nearest")
    plt.xlabel('frames')
    plt.ylabel('nodes')
    plt.yticks(np.arange(n_bodyparts), bodyparts, rotation=20);
    plt.title(f'Mouse {instance_idx} node velocities')

In [ ]:
mouse_ID = 0
mouse_node_velocities = instance_node_velocities(mouse_ID)
plot_instance_node_velocities(mouse_ID, mouse_node_velocities)

In [ ]:
mouse_ID = 1
mouse_node_velocities = instance_node_velocities(mouse_ID)
plot_instance_node_velocities(mouse_ID, mouse_node_velocities)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
nstates = 10

km = KMeans(n_clusters=nstates)

labels = km.fit_predict(mouse_node_velocities)

In [ ]:
fig = plt.figure(figsize=(20, 12))

ax1 = fig.add_subplot(211)
ax1.imshow(mouse_node_velocities.T, aspect="auto", vmin=0, vmax=20, interpolation="nearest")
ax1.set_xlabel("Frames")
ax1.set_ylabel("Nodes")
ax1.set_yticks(np.arange(n_bodyparts))
ax1.set_yticklabels(bodyparts);
ax1.set_title(f"Mouse {mouse_ID} node velocities")
ax1.set_xlim(0,n_frames)

ax2 = fig.add_subplot(212,sharex=ax1)
ax2.imshow(labels[None, :], aspect="auto", cmap="tab10", interpolation="nearest")
ax2.set_xlabel("Frames")
ax2.set_yticks([])
ax2.set_title("Ethogram (colors = clusters)");